In [ ]:
import pandas as pd
import numpy as np
import re
import datetime

pd.set_option('display.max_colwidth', -1)

In [2]:
inspections = pd.read_csv("prohlidky.csv")
inspections = inspections.drop(['Unnamed: 0'], axis=1)
inspections.DatKont = pd.to_datetime(inspections.DatKont)
inspections.DatPrvReg = pd.to_datetime(inspections.DatPrvReg)
display(inspections.head())
display(inspections.info())

,STK,DrTP,VIN,DatKont,TypMot,TZn,DrVoz,ObchOznTyp,Ct,DatPrvReg,Km,ZavA,ZavB,ZavC,VyslSTK,VyslEmise
0,3122,Evidencni kontrola,JMZBLA2G601258504,2018-01-02 11:03:12.833,LF,MAZDA,OSOBNI AUTOMOBIL,3,M1,2011-02-10,84818,0,0,0,zpusobile,---
1,3205,Evidencni kontrola,4150417,2018-01-02 11:06:07.617,EM150.2,MZ,MOTOCYKL,ETZ 150,L3e,1989-01-01,38828,0,0,0,zpusobile,---
2,3114,Evidencni kontrola,VF3MJAHXHGS280168,2018-01-02 11:15:08.083,AH01,PEUGEOT,OSOBNI AUTOMOBIL,3008,M1,2017-01-09,39227,0,0,0,zpusobile,---
3,3618,Evidencni kontrola,4699845,2018-01-02 11:19:22.967,1202,SKODA,OSOBNI AUTOMOBIL,1202,M1,1979-06-04,38951,0,0,0,zpusobile,---
4,3748,Evidencni kontrola,WF0SXXGCDSAU06730,2018-01-02 11:30:25.420,GPDC,FORD,OSOBNI AUTOMOBIL,FOCUS (DA3),M1,2010-06-29,254194,0,0,0,zpusobile,---


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3728369 entries, 0 to 3728368
Data columns (total 16 columns):
STK           int64
DrTP          object
VIN           object
DatKont       datetime64[ns]
TypMot        object
TZn           object
DrVoz         object
ObchOznTyp    object
Ct            object
DatPrvReg     datetime64[ns]
Km            int64
ZavA          int64
ZavB          int64
ZavC          int64
VyslSTK       object
VyslEmise     object
dtypes: datetime64[ns](2), int64(5), object(9)
memory usage: 455.1+ MB


None

In [3]:
stations = pd.read_csv("stk_form_pg.csv")
# stations = stations.drop(['Unnamed: 0'], axis=1)
display(stations.head())
display(stations.info())

,stk_id,rozsah_opravneni,psc,mesto,ulice,provozovatel,tel,mail,orp,okres,kraj,cenastk,cenaevkont,cenaemiseb,cenaemisen,pracovnidoba
0,312,"OA, ZS",198 00,Praha 14,Skorkovská 1537,"TÜV NORD Czech, s.r.o.","281 915 951, 281 915 952",zizkovsky@tuev-nord.cz; stk-praha@tuev-nord.cz,Praha,Praha,Hlavní město Praha,0,0,0,0,"Po: 7:30-19:30, Út: 7:30-19:30, St: 7:30-19:30, Čt: 7:30-19:30, Pá: 7:30-19:30, So: 8:00-16:00"
1,3123,"OA, ZS",148 25,Praha 4,Dobronická 1216,STK Kunratice spol. s r.o.,242456194,stkkunratice@gmail.cz,Praha,Praha,Hlavní město Praha,0,0,0,0,"Po: 6:00-15:00, Út: 6:00-14:30, St: 6:00-14:30, Čt: 6:00-14:30, Pá: 6:00-12:00"
2,3127,OA,184 00,Praha 9,Dopraváků 874/15,TEKOSO-STK spol.s r.o.,273130110,stk3127@seznam.cz,Praha,Praha,Hlavní město Praha,0,0,0,0,"Po: 7:00-17:00, Út: 7:00-17:00, St: 7:00-17:00, Čt: 7:00-17:00, Pá: 7:00-17:00"
3,3126,OA,149 00,Praha 4,Ohradní 1575,PRONTO STK s.r.o.,607691696,stkpraha@stkpraha.cz,Praha,Praha,Hlavní město Praha,0,0,0,0,"Po: 7:00-17:30, Út: 7:00-17:30, St: 7:00-17:30, Čt: 7:00-17:30"
4,3128,OA,149 00,Praha 4,Mezi vodami 1097/25,ESPM s.r.o.,725888170,info@stk-modrany.cz,Praha,praha,Hlavní město Praha,0,0,0,0,"Po: 7:30-17:00, Út: 7:30-17:00, St: 7:30-17:00, Čt: 7:30-17:00, Pá: 7:30-17:00"


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 372 entries, 0 to 371
Data columns (total 16 columns):
stk_id              372 non-null int64
rozsah_opravneni    372 non-null object
psc                 372 non-null object
mesto               372 non-null object
ulice               372 non-null object
provozovatel        372 non-null object
tel                 372 non-null object
mail                372 non-null object
orp                 372 non-null object
okres               372 non-null object
kraj                372 non-null object
cenastk             372 non-null int64
cenaevkont          372 non-null int64
cenaemiseb          372 non-null int64
cenaemisen          372 non-null int64
pracovnidoba        372 non-null object
dtypes: int64(5), object(11)
memory usage: 46.6+ KB


None

In [4]:
inspections_split = np.array_split(inspections, 1000)
out_of_working_hours = pd.DataFrame(data=None, columns=inspections.columns)
out_of_working_hours['pracovnidoba'] = np.nan
out_of_working_hours['duvod'] = np.nan

In [ ]:
stations.loc[stations['pracovnidoba'] == "Nákladní auta", 'pracovnidoba'] = "0"
stations.loc[stations['pracovnidoba'] == "Pouze Nákladní a autobusy", 'pracovnidoba'] = "0"
stations['stk_id'] = stations['stk_id'].apply(lambda x: x if 4 == len(str(x)) else x * 10)
daysofweek = [1, 2, 3, 4, 5, 6, 0]
daysofweekstr = {
  0: "Po: ",
  1: "Út: ",
  2: "St: ",
  3: "Čt: ",
  4: "Pá: ",
  5: "So: ",
  6: "Ne: "
}

for part in inspections_split:
    out_of_working_hours = out_of_working_hours.iloc[0:0]
    for index, row in part.iterrows():
        for dayofweek in daysofweek:
            if row.DatKont.weekday() == dayofweek:
                try:
                    if len(stations.where(stations['stk_id'] == row['STK']).dropna(how='all').pracovnidoba.all()) > 10:
                        pracovnidoba = stations.where(stations['stk_id'] == row['STK']).dropna(how='all').pracovnidoba.to_string()
                        index = pracovnidoba.find(daysofweekstr[dayofweek])
                        if index != -1:
                            pattern = re.compile('(.)*{0}(\d):(\d\d)+-(\d\d):(\d\d).*'.format(daysofweekstr[dayofweek]))
                            match = pattern.match(str(pracovnidoba))
                            if match:
                                insp_time = row['DatKont'].hour * 60 + row['DatKont'].minute
                                before = int(match.group(2)) * 60 + int(match.group(3))
                                after = int(match.group(4)) * 60 + int(match.group(5))
                                if insp_time < before:
                                    row['pracovnidoba'] = pracovnidoba
                                    row['duvod'] = "OUT OF TIME RANGE: -{0}".format(before - insp_time)
                                    out_of_working_hours = out_of_working_hours.append(row)
                                if insp_time > after:
                                    row['pracovnidoba'] = pracovnidoba
                                    row['duvod'] = "OUT OF TIME RANGE: +{0}".format(insp_time - after)
                                    out_of_working_hours = out_of_working_hours.append(row)
                            else:
                                row['pracovnidoba'] = pracovnidoba
                                row['duvod'] = "ERROR: NO MATCH"
                                out_of_working_hours = out_of_working_hours.append(row)
                        else:
                            row['pracovnidoba'] = pracovnidoba
                            row['duvod'] = "OUT OF WEEKDAY: {0}".format(row.DatKont.weekday())
                            out_of_working_hours = out_of_working_hours.append(row)
                except:
                    print(row)
    out_of_working_hours.to_csv("out_of_working_hours.csv", mode='a')

In [9]:
out_of_working_hours = pd.read_csv('out_of_working_hours_2.csv')

In [20]:
out_of_working_hours = out_of_working_hours.drop_duplicates()
out_of_working_hours.where(out_of_working_hours['duvod'].str.match(r'OUT OF TIME RANGE: +')).dropna(how='all').groupby('STK').count()

,Unnamed: 0,DrTP,VIN,DatKont,TypMot,TZn,DrVoz,ObchOznTyp,Ct,DatPrvReg,Km,ZavA,ZavB,ZavC,VyslSTK,VyslEmise,pracovnidoba,duvod
STK,,,,,,,,,,,,,,,,,,
3100,79,79,79,79,76,79,79,79,79,79,79,79,79,79,79,79,79,79
3102,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47
3104,1316,1316,1316,1316,1297,1316,1316,1316,1316,1316,1316,1316,1316,1316,1316,1316,1316,1316
3105,1133,1133,1133,1133,1121,1133,1133,1133,1133,1133,1133,1133,1133,1133,1133,1133,1133,1133
3107,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3847,9,9,9,9,7,9,9,9,9,9,9,9,9,9,9,9,9,9
3850,143,143,143,143,139,143,143,143,143,143,143,143,143,143,143,143,143,143
3851,2236,2236,2236,2236,2213,2236,2236,2236,2236,2236,2236,2236,2236,2236,2236,2236,2236,2236


In [ ]:
# stations['Po'] = np.nan
# stations['Ut'] = np.nan
# stations['St'] = np.nan
# stations['Ct'] = np.nan
# stations['Pa'] = np.nan
# stations['So'] = np.nan
# stations['Ne'] = np.nan

# stations['pracovniDoba'] = stations['pracovniDoba'].astype(str).str.replace('Provozní doba','')
# stations['pracovniDoba'] = stations['pracovniDoba'].astype(str).str.replace('Na objednávku','')
# stations['pracovniDoba'] = stations['pracovniDoba'].astype(str).str.replace('Pracovní doba:','')
# stations['pracovniDoba'] = stations['pracovniDoba'].astype(str).str.replace('Po předchozí tel. objednávce','')
# stations['pracovniDoba'] = stations['pracovniDoba'].astype(str).str.replace('Nákladní auta > Dobronická 635\n','')
# stations['pracovniDoba'] = stations['pracovniDoba'].astype(str).str.replace('\r','')
# stations['pracovniDoba'] = stations['pracovniDoba'].astype(str).str.replace('\t','')
# stations['pracovniDoba'] = stations['pracovniDoba'].astype(str).str.replace('\n','')

# for index, row in stations.iterrows():
#     tmp = row['pracovniDoba'].replace(' ', '')
#     if row['pracovniDoba'] == '-':
#             stations.loc[index, 'Po'] = '-'
#             stations.loc[index, 'Ut'] = '-'
#             stations.loc[index, 'St'] = '-'
#             stations.loc[index, 'Ct'] = '-'
#             stations.loc[index, 'Pa'] = '-'
#             stations.loc[index, 'So'] = '-'
#             stations.loc[index, 'Ne'] = '-'
#     if re.match(r'.*Po-Pá:.*', tmp):
#         tmp = tmp[tmp.index('Po-Pá:') + len('Po-Pá:'):]
#         if re.match(r'\d(:|.)\d\d-\d\d(:|.)\d\d.*', tmp):
#             stations.loc[index, 'Po'] = tmp[:10]
#             stations.loc[index, 'Ut'] = tmp[:10]
#             stations.loc[index, 'St'] = tmp[:10]
#             stations.loc[index, 'Ct'] = tmp[:10]
#             stations.loc[index, 'Pa'] = tmp[:10]
            
#             if re.match(r'.*So:.*', tmp[10:]):
#                 tmp = tmp[tmp.index('So:') + len('So:'):]
#                 stations.loc[index, 'So'] = tmp[:10]
            
#             if re.match(r'.*Ne:.*', tmp[10:]):
#                 tmp = tmp[tmp.index('Ne:') + len('Ne:'):]
#                 stations.loc[index, 'Ne'] = tmp[:10]
                
#         if re.match(r'\d-\d\d.*', tmp):
#             stations.loc[index, 'Po'] = tmp[:4]
#             stations.loc[index, 'Ut'] = tmp[:4]
#             stations.loc[index, 'St'] = tmp[:4]
#             stations.loc[index, 'Ct'] = tmp[:4]
#             stations.loc[index, 'Pa'] = tmp[:4]
            
#             if re.match(r'.*So:.*', tmp[4:]):
#                 tmp = tmp[tmp.index('So:') + len('So:'):]
#                 stations.loc[index, 'So'] = tmp[:4]
                
#             if re.match(r'.*Ne:.*', tmp[4:]):
#                 tmp = tmp[tmp.index('Ne:') + len('Ne:'):]
#                 stations.loc[index, 'Ne'] = tmp[:4]
                
#     tmp = row['pracovniDoba'].replace(' ', '')      
#     if re.match(r'.*Po-Čt:.*', tmp):
#         tmp = tmp[tmp.index('Po-Čt:') + len('Po-Čt:'):]
#         if re.match(r'\d(:|.)\d\d-\d\d(:|.)\d\d.*', tmp):
#             stations.loc[index, 'Po'] = tmp[:10]
#             stations.loc[index, 'Ut'] = tmp[:10]
#             stations.loc[index, 'St'] = tmp[:10]
#             stations.loc[index, 'Ct'] = tmp[:10]
            
#             if re.match(r'.*Pá:.*', tmp[10:]):
#                 tmp = tmp[tmp.index('Pá:') + len('Pá:'):]
#                 stations.loc[index, 'Pa'] = tmp[:10]
            
#             if re.match(r'.*So:.*', tmp[10:]):
#                 tmp = tmp[tmp.index('So:') + len('So:'):]
#                 stations.loc[index, 'So'] = tmp[:10]
                
#             if re.match(r'.*Ne:.*', tmp[10:]):
#                 tmp = tmp[tmp.index('Ne:') + len('Ne:'):]
#                 stations.loc[index, 'Ne'] = tmp[:10]
            
#         if re.match(r'\d-\d\d.*', tmp):
#             stations.loc[index, 'Po'] = tmp[:4]
#             stations.loc[index, 'Ut'] = tmp[:4]
#             stations.loc[index, 'St'] = tmp[:4]
#             stations.loc[index, 'Ct'] = tmp[:4]
            
#             if re.match(r'.*Pá:.*', tmp[4:]):
#                 tmp = tmp[tmp.index('Pá:') + len('Pá:'):]
#                 stations.loc[index, 'Pa'] = tmp[:4]
            
#             if re.match(r'.*So:.*', tmp[4:]):
#                 tmp = tmp[tmp.index('So:') + len('So:'):]
#                 stations.loc[index, 'So'] = tmp[:4]
                
#             if re.match(r'.*Ne:.*', tmp[4:]):
#                 tmp = tmp[tmp.index('Ne:') + len('Ne:'):]
#                 stations.loc[index, 'Ne'] = tmp[:4]
                
#     tmp = row['pracovniDoba'].replace(' ', '')               
#     if re.match(r'.*Po,St:.*', tmp):
#         tmp = tmp[tmp.index('Po,St:') + len('Po,St:'):]
#         if re.match(r'\d(:|.)\d\d-\d\d(:|.)\d\d.*', tmp):
#             stations.loc[index, 'Po'] = tmp[:10]
#             stations.loc[index, 'St'] = tmp[:10]
            
#             if re.match(r'.*Pá:.*', tmp[10:]):
#                 tmp = tmp[tmp.index('Pá:') + len('Pá:'):]
#                 stations.loc[index, 'Pa'] = tmp[:10]
            
#             if re.match(r'.*So:.*', tmp[10:]):
#                 tmp = tmp[tmp.index('So:') + len('So:'):]
#                 stations.loc[index, 'So'] = tmp[:10]
                
#             if re.match(r'.*Ne:.*', tmp[10:]):
#                 tmp = tmp[tmp.index('Ne:') + len('Ne:'):]
#                 stations.loc[index, 'Ne'] = tmp[:10]
            
#         if re.match(r'\d-\d\d.*', tmp):
#             stations.loc[index, 'Po'] = tmp[:4]
#             stations.loc[index, 'St'] = tmp[:4]
            
#             if re.match(r'.*Pá:.*', tmp[4:]):
#                 tmp = tmp[tmp.index('Pá:') + len('Pá:'):]
#                 stations.loc[index, 'Pa'] = tmp[:4]
            
#             if re.match(r'.*So:.*', tmp[4:]):
#                 tmp = tmp[tmp.index('So:') + len('So:'):]
#                 stations.loc[index, 'So'] = tmp[:4]
                
#             if re.match(r'.*Ne:.*', tmp[4:]):
#                 tmp = tmp[tmp.index('Ne:') + len('Ne:'):]
#                 stations.loc[index, 'Ne'] = tmp[:4]
                
#     tmp = row['pracovniDoba'].replace(' ', '')                
#     if re.match(r'.*Út,Čt:.*', tmp):
#         tmp = tmp[tmp.index('Út,Čt:') + len('Út,Čt:'):]
#         if re.match(r'\d(:|.)\d\d-\d\d(:|.)\d\d.*', tmp):
#             stations.loc[index, 'Ut'] = tmp[:10]
#             stations.loc[index, 'Ct'] = tmp[:10]
            
#             if re.match(r'.*Pá:.*', tmp[10:]):
#                 tmp = tmp[tmp.index('Pá:') + len('Pá:'):]
#                 stations.loc[index, 'Pa'] = tmp[:10]
            
#             if re.match(r'.*So:.*', tmp[10:]):
#                 tmp = tmp[tmp.index('So:') + len('So:'):]
#                 stations.loc[index, 'So'] = tmp[:10]
                
#             if re.match(r'.*Ne:.*', tmp[10:]):
#                 tmp = tmp[tmp.index('Ne:') + len('Ne:'):]
#                 stations.loc[index, 'Ne'] = tmp[:10]
            
#         if re.match(r'\d-\d\d.*', tmp):
#             stations.loc[index, 'Ut'] = tmp[:4]
#             stations.loc[index, 'Ct'] = tmp[:4]
            
#             if re.match(r'.*Pá:.*', tmp[4:]):
#                 tmp = tmp[tmp.index('Pá:') + len('Pá:'):]
#                 stations.loc[index, 'Pa'] = tmp[:4]
            
#             if re.match(r'.*So:.*', tmp[4:]):
#                 tmp = tmp[tmp.index('So:') + len('So:'):]
#                 stations.loc[index, 'So'] = tmp[:4]
                
#             if re.match(r'.*Ne:.*', tmp[4:]):
#                 tmp = tmp[tmp.index('Ne:') + len('Ne:'):]
#                 stations.loc[index, 'Ne'] = tmp[:4]
                
#     tmp = row['pracovniDoba'].replace(' ', '')                
#     if re.match(r'\d(:|.)\d\d-\d\d(:|.)\d\d.*', tmp):
#         if re.match(r'.*Po:.*', tmp[10:]):
#             tmp = tmp[tmp.index('Po:') + len('Po:'):]
#             stations.loc[index, 'Po'] = tmp[:10]

#         if re.match(r'.*Út:.*', tmp[10:]):
#             tmp = tmp[tmp.index('Út:') + len('Út:'):]
#             stations.loc[index, 'Út'] = tmp[:10]
            
#         if re.match(r'.*St:.*', tmp[10:]):
#             tmp = tmp[tmp.index('St:') + len('St:'):]
#             stations.loc[index, 'St'] = tmp[:10]
        
#         if re.match(r'.*Čt:.*', tmp[10:]):
#             tmp = tmp[tmp.index('Čt:') + len('Čt:'):]
#             stations.loc[index, 'Čt'] = tmp[:10]
            
#         if re.match(r'.*Pá:.*', tmp[10:]):
#             tmp = tmp[tmp.index('Pá:') + len('Pá:'):]
#             stations.loc[index, 'Pa'] = tmp[:10]

#         if re.match(r'.*So:.*', tmp[10:]):
#             tmp = tmp[tmp.index('So:') + len('So:'):]
#             stations.loc[index, 'So'] = tmp[:10]

#         if re.match(r'.*Ne:.*', tmp[10:]):
#             tmp = tmp[tmp.index('Ne:') + len('Ne:'):]
#             stations.loc[index, 'Ne'] = tmp[:10]
            
#     tmp = row['pracovniDoba'].replace(' ', '')
#     if re.match(r'\d-\d\d.*', tmp):
#         if re.match(r'.*Po:.*', tmp[4:]):
#             tmp = tmp[tmp.index('Po:') + len('Po:'):]
#             stations.loc[index, 'Po'] = tmp[:4]

#         if re.match(r'.*Út:.*', tmp[4:]):
#             tmp = tmp[tmp.index('Út:') + len('Út:'):]
#             stations.loc[index, 'Ut'] = tmp[:4]
            
#         if re.match(r'.*St:.*', tmp[4:]):
#             tmp = tmp[tmp.index('St:') + len('St:'):]
#             stations.loc[index, 'St'] = tmp[:4]
        
#         if re.match(r'.*Čt:.*', tmp[4:]):
#             tmp = tmp[tmp.index('Čt:') + len('Čt:'):]
#             stations.loc[index, 'Ct'] = tmp[:4]
#         if re.match(r'.*Pá:.*', tmp[4:]):
#             tmp = tmp[tmp.index('Pá:') + len('Pá:'):]
#             stations.loc[index, 'Pa'] = tmp[:4]

#         if re.match(r'.*So:.*', tmp[4:]):
#             tmp = tmp[tmp.index('So:') + len('So:'):]
#             stations.loc[index, 'So'] = tmp[:4]

#         if re.match(r'.*Ne:.*', tmp[4:]):
#             tmp = tmp[tmp.index('Ne:') + len('Ne:'):]
#             stations.loc[index, 'Ne'] = tmp[:4]
            

# stations['Po'] = stations['Po'].astype(str).str.replace(':00','')
# stations['Ut'] = stations['Ut'].astype(str).str.replace(':00','')
# stations['St'] = stations['St'].astype(str).str.replace(':00','')
# stations['Ct'] = stations['Ct'].astype(str).str.replace(':00','')
# stations['Pa'] = stations['Pa'].astype(str).str.replace(':00','')
# stations['So'] = stations['So'].astype(str).str.replace(':00','')
# stations['Ne'] = stations['Ne'].astype(str).str.replace(':00','')